In [1]:
import numpy as np
import math as math
import matplotlib.pyplot as plt

In [ ]:
def rk4(x, t, tau, derivsRK, param):     #Garcia's rk4 function
    """
    Runge-Kutta integrator (4th order)
    Input arguments
    :param x: current value of dependent variable
    :param t: independent variable (usually time)
    :param tau: step size (usually time step)
    :param derivsRK: right hand side of the ODE; derivsRK is the name of the function which returns dx/dt
    Calling format derivsRK (x, t, param).
    :param param: estra parameters passed to derivsRK  GM in this case
    :return:
    xout: new value of x after a step of size tau
    """

    half_tau = 0.5*tau
    F1 = derivsRK(x, t, param)
    t_half = t + half_tau
    xtemp = x + half_tau*F1
    F2 = derivsRK(xtemp, t_half, param)
    xtemp = x + half_tau*F2
    F3 = derivsRK(xtemp, t_half, param)
    t_full = t + tau
    xtemp = x + tau*F3
    F4 = derivsRK(xtemp, t_full, param)
    xout = x + tau/6.0 * (F1 + F4 + 2.0*(F2+F3))

    return xout

def LL_rk(s, t, gamma, M_mag, H, theta, alpha):       #Landau-Lifshitz Equation         
    """ 
    Returns the right-hand side of the Kepler ODE; used by Runge-Kutta routines
    :param s: State vector [r(0), r(1), v(0), v(1)]
    :param t: Time (not used here, included to match derivsRK input)
    :param GM: Parameter G*M - gravitational constant * solar mass Units [AU^3/yr^2]
    :return: deriv: Derivatives [dr(0)/dt), dr(1)/dt, dv(0)/dt, dv(1)/dt]
    """

    # Compute dM/dt
    r = s[:3]  # Unravel the vector s into position and velocity
    v = s[3:]
    #accel = -GM * r / np.linalg.norm(r)**3  #Keplerian Gravitational acceleration
    torque = -gamma*np.cross(r, H)
    damping = -alpha/M_mag * np.cross(r, torque)
    
    # Return derivatives
    deriv = torque - damping

    return deriv

def dinput(input_text) :           #text input functions
    return int(input(input_text))

def finput(input_text) :
    return float(input(input_text))

#Declare/Input physical paramters
M_mag = finput("Please input the magnitude of the Magnetization vector in [units]:")
theta_0 = finput("Please input the initial angle between the maagnetization vector and z-axis in radians:")
H_mag = finput("Please input the magnitude of the applied field in [units]:")
gamma = finput("Please input the gyromagnetic ratio for this material [units]:")
alpha = finput("please input the damping factor coefficient:")

M_vector = np.array([M_mag*sin(theta_0), 0.0, M_mag*cos(theta_0)])
dM = np.array([0.0, 0.0, 0.0])      #needs to be edited
state = np.array([M_vector[0], M_vector[1], , M_vector[2], dM[0], dM[1], dM[2]])
H_eff = np.array([0.0, 0.0, H_mag])

freq_res = -gamma*H_mag     #calculate Resonant frequency

time = 0.0
dt = .005
nStep = 1000

#the meat of the problems
for iStep in range(nStep):

    # Record position and energy for plotting
    rplot[iStep] = np.sqrt(M_vector[0]**2 + M_vector[1]**2)  # Record radial position and angle for polar plot
    phiplot[iStep] = np.arctan2(M_vector[1], M_vector[0])
    tplot[iStep] = time
    
    # Calculate new position and velocity using the Runge-Kutta method
    state = rk4(state, time, tau, LL_rk, gamma, M_mag, H, theta, alpha)
    M_vector = state[:3]  # 4th Order Runge-Kutta
    dM = state[3:]
    time += tau
